## 41. 数据流中的中位数

如何得到一个数据流中的中位数？如果从数据流中读出奇数个数值，那么中位数就是所有数值排序之后位于中间的数值。如果从数据流中读出偶数个数值，那么中位数就是所有数值排序之后中间两个数的平均值。我们使用Insert()方法读取数据流，使用GetMedian()方法获取当前读取数据的中位数。

### 分析
由于数据是从一个数据流中读出来的，因而数据的数目随着时间的变化而增加。如果用一个数据容器来保存从流中读出来的数据，则应当考虑一个合适的数据结构来存储。

- **数组**：可以像39题一样，每插入一个新的数据O(1)，都用partition函数找出数组中的中位数O(n)。

  
- **排序链表**：需要O(n)时间才能在链表中找到合适的位置插入新的数据。如果定义两个指针指向链表中间的节点（如果链表的节点数目是奇数，那么这两个指针指向同一个节点），那么可以在O(1)时间内得出中位数 --> 和排序数组的时间差不多。


- **二叉搜索树**：可以把插入新数据的平均时间降低到O(log n)。但是二叉树有可能变成极度不平衡从而看起来像一个排序的链表，这样插入新数据的时间任然是O(n)。为了得到中位数，可以在二叉树节点中添加一个表示子树节点数目的字段。有了这个字段，可以在平均O(log n)时间内得到中位数，但最差情况仍需要O(n)时间。


- **AVL树**：为了避免二叉搜索树的最差情况，还可以利用平衡的二叉搜索树，[AVL树](https://zhuanlan.zhihu.com/p/34899732)。通常AVL树的平衡因子是左，右子树节点高度之差。可以把这个平衡条件更改为左右子树节点数目之差。这样可以用O(log n)时间往AVL树中添加一个新节点，同时用O(1)时间得到所有节点的中位数。此方法效率很高，但是大部分语言中并没有AVL树的实现，临场写也不太可能。


- **最大最小堆**：如下图所示，如果数据在容器中已经排好序，那么中位数可以由P1和P2指向的数得到。
<img src="images/img41_a.png" style="width: 450px;"/>
注意到如果把数据分成两部分，P1指向的是左边部分最大的树，P2指向的是右边部分最小的数。如果能保证这个特性，那么左右两边内部的数据即使没有排序，也可找到中位数。我们可以用max heap实现左边部分，min heap实现右边部分。这样插入数据的效率是O(log n)，取得位于heap顶的数据是O(1)。
<img src="images/img41_b.png" style="width: 460px;"/>
<img src="images/img41_c.png" style="width: 450px;"/>

[//]: # (<img src="images/img123.png" style="width: 400px;"/>)

### 实现最大堆和最小堆的细节

1. 首先应保证数据平均分配到两个堆中，因此两个堆中数据的数目之差不能超过1。为了实现平均分配，可以在数据的总数目是偶数时把新数据插入min heap（右），否则插入max heap（左）。

2. 保证min heap的所有数据都大于max heap的所有数据。当数据总数为偶数时，按照前面的规则会把新的数据插入min heap（右），如果这个新数据比max heap（左）的top要小，则需要把新数据先插入max heap（左），把最大的数拿出来插入min heap（右）。反之亦然。

In [1]:
import heapq

class Solution:
    def __init__(self):
        self.maxheap = []
        self.minheap = []
        heapq.heapify(self.minheap)
        heapq._heapify_max(self.maxheap)
        
    def Insert(self, num):
        if num is None or not isinstance(num, int):
            return
    
        if (len(self.maxheap) + len(self.minheap)) & 1 == 0: # if total length is even
            if len(self.maxheap) > 0:
                current_max = heapq._heappop_max(self.maxheap)
                if num >= current_max:
                    # insert num in minheap
                    heapq.heappush(self.minheap, num)
                    heapq.heappush(self.maxheap, current_max)
                    heapq._heapify_max(self.maxheap)
                else:
                    # insert num in maxheap, store current_max in minheap
                    heapq.heappush(self.maxheap, num)
                    heapq._heapify_max(self.maxheap)
                    heapq.heappush(self.minheap, current_max)
            else:
                heapq.heappush(self.minheap, num)
        else:
            if len(self.minheap) > 0:       
                current_min = heapq.heappop(self.minheap)
                if num <= current_min:
                    # insert num in maxheap
                    heapq.heappush(self.maxheap, num)
                    heapq._heapify_max(self.maxheap)
                    heapq.heappush(self.minheap, current_min)
                else:
                    # insert num in minheap, store current_min in maxheap
                    heapq.heappush(self.minheap, num)
                    heapq.heappush(self.maxheap, current_min)
                    heapq._heapify_max(self.maxheap)
            else:
                heapq.heappush(self.maxheap, num)
                heapq._heapify_max(self.maxheap)
        
    def GetMedian(self):
        total_len = len(self.maxheap) + len(self.minheap)
        if total_len == 0:
            raise Exception('Length of array is 0')

        median = 0
        if total_len & 1 == 0: # even number
            median = (self.maxheap[0] + self.minheap[0]) / 2
        else:
            median = self.minheap[0]

        return median

In [2]:
# Test
solution = Solution()
solution.Insert(5)
print("current median: {}".format(solution.GetMedian()))
solution.Insert(2)
print("current median: {}".format(solution.GetMedian()))
solution.Insert(3)
print("current median: {}".format(solution.GetMedian()))
solution.Insert(4)
print("left:  {}".format(solution.maxheap))
print("right: {}".format(solution.minheap))
print("current median: {}".format(solution.GetMedian()))
solution.Insert(1)
print("current median: {}".format(solution.GetMedian()))
solution.Insert(6)
print("current median: {}".format(solution.GetMedian()))
solution.Insert(7)
print("current median: {}".format(solution.GetMedian()))
solution.Insert(0)
print("current median: {}".format(solution.GetMedian()))
solution.Insert(8)
print("left:  {}".format(solution.maxheap))
print("right: {}".format(solution.minheap))
print("current median: {}".format(solution.GetMedian()))

# print("left:  {}".format(solution.maxheap))
# print("right: {}".format(solution.minheap))
# print("current median: {}".format(solution.GetMedian()))

current median: 5
current median: 3.5
current median: 3
left:  [3, 2]
right: [4, 5]
current median: 3.5
current median: 3
current median: 3.5
current median: 4
current median: 3.5
left:  [3, 2, 0, 1]
right: [4, 5, 6, 7, 8]
current median: 4


### Max/min Heap in Python

```python
import heapq
############
# min heap #
############
minheap = [1, 2, 3, 4, 5, 6, 7, 8, 9]    
heapq.heapify(minheap)       
# Get current min
current_min = heapq.heappop(minheap) 
# Push new element in minheap
heapq.heappush(minheap, 10)

############
# max heap #
############
maxheap = [1, 2, 3, 4, 5, 6, 7, 8, 9] 
heapq._heapify_max(maxheap)
# Get current max
current_max = heapq._heappop_max(maxheap)
# Push new element in maxheap, but you need to rearrange it to a maxheap
heapq.heappush(minheap, 10)
heapq._heapify_max(maxheap)
```